# Big Data Analysis Homework 2024-09-16

In [1]:
import polars as pl
import altair as alt
from functools import reduce

載入資料。

In [2]:
df = pl.read_csv("missing-data.csv")
df

crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
f64,f64,f64,i64,f64,f64,f64,f64,i64,i64,f64,f64,f64,f64
0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296,15.3,396.9,4.98,24.0
null,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.9,9.14,21.6
0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.9,5.33,36.2
…,…,…,…,…,…,…,…,…,…,…,…,…,…
0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
0.04527,0.0,11.93,0,0.573,6.12,76.7,2.2875,1,273,21.0,396.9,9.08,20.6
0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.9,5.64,23.9


## 探索性分析

## 變數說明

The original dataset is [The Boston Housing Dataset](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html).

- `CRIM` - 每個城鎮的人均犯罪率
  - 連續尺度
- `ZN` - 25,000 平方英尺以上的住宅用地比例
  - 連續尺度
- `INDUS` - 每個城鎮的非零售業用地比例
  - 連續尺度
- `CHAS` - 查爾斯河虛擬變數（如果區域邊界靠近河流則為 1，否則為 0）
  - 名目尺度，二元虛擬變數
- `NOX` - 一氧化氮濃度（每千萬分之一）
  - 連續尺度
- `RM` - 每戶平均房間數
  - 連續尺度
- `AGE` - 1940 年前建造的自有住宅單位比例
  - 連續尺度
- `DIS` - 到五個波士頓就業中心的加權距離
  - 連續尺度
- `RAD` - 徑向高速公路可達性的指數
  - 順序尺度
- `TAX` - 每\$10,000 的全值財產稅率
  - 連續尺度
- `PTRATIO` - 每個城鎮的師生比例
  - 連續尺度
- `B` - 1000(Bk - 0.63)^2，其中 Bk 是每個城鎮的黑人比例
  - 連續尺度
- `LSTAT` - 低社會經濟地位人口的百分比
  - 連續尺度
- `MEDV` - 自有住宅的中位數價值（以\$1000 計算）
  - 連續尺度

### 資料描述

In [3]:
df.describe()

statistic,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",500.0,506.0,496.0,506.0,502.0,504.0,502.0,501.0,506.0,501.0,501.0,501.0,502.0,502.0
"""null_count""",6.0,0.0,10.0,0.0,4.0,2.0,4.0,5.0,0.0,5.0,5.0,5.0,4.0,4.0
"""mean""",3.652345,11.363636,11.102601,0.06917,0.554426,6.284177,68.597809,3.78875,9.549407,408.952096,18.448303,356.6002,12.670179,22.560159
"""std""",8.645644,23.322453,6.837721,0.253994,0.115286,0.703974,28.182902,2.107677,8.707259,168.97714,2.169678,91.563468,7.155505,9.218467
"""min""",0.00632,0.0,0.46,0.0,0.385,3.561,2.9,1.1296,1.0,187.0,12.6,0.32,1.73,5.0
"""25%""",0.08199,0.0,5.19,0.0,0.449,5.885,45.0,2.1,4.0,279.0,17.3,375.33,6.93,17.0
"""50%""",0.26169,0.0,9.69,0.0,0.538,6.208,77.7,3.1827,5.0,330.0,19.1,391.43,11.38,21.2
"""75%""",3.69311,12.5,18.1,0.0,0.624,6.625,94.1,5.118,24.0,666.0,20.2,396.23,17.09,25.0
"""max""",88.9762,100.0,27.74,1.0,0.871,8.78,100.0,12.1265,24.0,711.0,22.0,396.9,37.97,50.0


其中 `count` 是非空資料筆數，`null_count` 是空資料筆數，`mean` 是平均數，`std` 是標準差，`min` - `max` 分別為最小值、四分位差和最大值。

對屬性資料畫出長條圖。

In [4]:
(
    alt.Chart(df["chas"].to_frame()).mark_bar().encode(x="chas:N", y='count()').interactive()
    | alt.Chart(df["rad"].to_frame()).mark_bar().encode(x="rad:N", y='count()').interactive()
)

alt.HConcatChart(...)

然後對數值資料列出其偏值和峰度。

In [5]:
with pl.Config(tbl_rows=-1):
    display(pl.DataFrame(
        {
            "column": df.columns,
            "skewness": [df[col].skew() for col in df.columns],
            "kurtosis": [df[col].kurtosis() for col in df.columns],
        }
    ))

column,skewness,kurtosis
str,f64,f64
"""crim""",5.177267,36.323694
"""zn""",2.219063,3.979949
"""indus""",0.28999,-1.237824
"""chas""",3.395799,9.531453
"""nox""",0.712946,-0.100768
"""rm""",0.403619,1.843919
"""age""",-0.600138,-0.969559
"""dis""",1.019548,0.49316
"""rad""",1.001833,-0.870521


然後對數值資料畫出直方圖及其常態分佈曲線。

In [6]:
df_ = df.clone().drop("chas", "rad")

histograms: list[alt.HConcatChart | alt.ConcatChart] = []

for col in df_.columns:
    hist = alt.Chart(df_[col].to_frame()).mark_bar().encode(
        alt.X(f"{col}:Q", bin=alt.Bin(maxbins=50)),
        y='count()',
    )

    density = alt.Chart(df_[col].to_frame()).transform_density(
        f"{col}",
        as_=[f"{col}", "density"],
    ).mark_area().encode(
        alt.X(f"{col}:Q"),
        alt.Y("density:Q"),
    )

    histograms.append(hist | density)

reduce(lambda x, y: x & y, histograms)

alt.VConcatChart(...)

可以發現這些變數可能是個常態分配 ($\left|\text{skewness}\right| < 3$)

| 表格 | 分佈 |
|---|----|
| crim | 不是常態分佈（偏度大於 3）|
| zn | 不太是常態分佈 |
| indus | 雙峰分佈 |
| nox | 右偏常態分佈 |
| rm | 接近完美的正態常態分佈 |
| age | 左偏常態分佈 |
| dis | 右偏常態分佈 |
| tax | 右偏常態分佈 |
| ptratio | 左偏常態分佈 |
| b | 不太是常態分佈 |
| lstat | 右偏常態分佈 |
| medv | 右偏常態分佈 |

故針對資料，可以得出這些結論：

- `chas` 可以看出不靠近河流的地區是多數
- `rad`（高速公路可達性）可以看出在高速公路附近 ($\text{rad}<24$) 的城市約佔據 50%~75%
- `crim` 和 `zn` 都是一個比例，照理來說應該是 $0$ 到 $1$ 的數值，但實際上卻有超出 $1$ 的數值，有待確認。
- `indus` 是個雙峰分配，在 $0$ 到 $14$ 及 $18$ 到 $22$ 有兩個高峰。
- `nox`、`dis`、`tax`、`lstat`、`medv` 均為右偏常態分佈
- `age`、`ptratio` 均為左偏常態分佈
- `rm` 是接近完美的常態分佈

# 資料清理

In [7]:
df.describe()

statistic,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",500.0,506.0,496.0,506.0,502.0,504.0,502.0,501.0,506.0,501.0,501.0,501.0,502.0,502.0
"""null_count""",6.0,0.0,10.0,0.0,4.0,2.0,4.0,5.0,0.0,5.0,5.0,5.0,4.0,4.0
"""mean""",3.652345,11.363636,11.102601,0.06917,0.554426,6.284177,68.597809,3.78875,9.549407,408.952096,18.448303,356.6002,12.670179,22.560159
"""std""",8.645644,23.322453,6.837721,0.253994,0.115286,0.703974,28.182902,2.107677,8.707259,168.97714,2.169678,91.563468,7.155505,9.218467
"""min""",0.00632,0.0,0.46,0.0,0.385,3.561,2.9,1.1296,1.0,187.0,12.6,0.32,1.73,5.0
"""25%""",0.08199,0.0,5.19,0.0,0.449,5.885,45.0,2.1,4.0,279.0,17.3,375.33,6.93,17.0
"""50%""",0.26169,0.0,9.69,0.0,0.538,6.208,77.7,3.1827,5.0,330.0,19.1,391.43,11.38,21.2
"""75%""",3.69311,12.5,18.1,0.0,0.624,6.625,94.1,5.118,24.0,666.0,20.2,396.23,17.09,25.0
"""max""",88.9762,100.0,27.74,1.0,0.871,8.78,100.0,12.1265,24.0,711.0,22.0,396.9,37.97,50.0


根據我們的第一點觀察，`chas` 應以 `0` 居多。或許我們可以試試看把 `chas` 都補 `0` 進去。

In [8]:
(
    alt.Chart(df["chas"].to_frame()).mark_bar().encode(x="chas:N", y='count()').properties(title="補 0 前").interactive() |
    alt.Chart(df["chas"].fill_null(0).to_frame()).mark_bar().encode(x="chas:N", y='count()').properties(title="補 0 後").interactive()
)

alt.HConcatChart(...)

針對單峰常態分配的 `nox`, `dis`, `tax`, `lstat`, `medv`, `age`, `ptratio`, `rm`，我們則取他的中位數來補空值，盡量保持分配的形狀。

In [9]:
stdcol = ["nox", "dis", "tax", "lstat", "medv", "age", "ptratio", "rm"]

# calculate its median
df_ = df.clone().select(stdcol)
df_ = df_.median()

new_df = df.clone()

for col in stdcol:
    new_df = new_df.with_columns(
        pl.col(col).fill_null(df_[col])
    )

In [10]:
new_df.describe()

statistic,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",500.0,506.0,496.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,501.0,506.0,506.0
"""null_count""",6.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
"""mean""",3.652345,11.363636,11.102601,0.06917,0.554296,6.283864,68.669763,3.782762,9.549407,408.171937,18.454743,356.6002,12.659822,22.549407
"""std""",8.645644,23.322453,6.837721,0.253994,0.114837,0.702596,28.082658,2.098075,8.707259,168.320157,2.159874,91.563468,7.128057,9.182677
"""min""",0.00632,0.0,0.46,0.0,0.385,3.561,2.9,1.1296,1.0,187.0,12.6,0.32,1.73,5.0
"""25%""",0.08199,0.0,5.19,0.0,0.449,5.885,45.1,2.1007,4.0,280.0,17.4,375.33,7.01,17.1
"""50%""",0.26169,0.0,9.69,0.0,0.538,6.205,77.7,3.1827,5.0,330.0,19.1,391.43,11.36,21.2
"""75%""",3.69311,12.5,18.1,0.0,0.624,6.625,94.0,5.118,24.0,666.0,20.2,396.23,16.96,25.0
"""max""",88.9762,100.0,27.74,1.0,0.871,8.78,100.0,12.1265,24.0,711.0,22.0,396.9,37.97,50.0


雙峰分配的 `indus`，我們則是把一半的值填左峰 (Q2)，另一半填右峰 (Q3)。

In [11]:
indus_left_q = new_df["indus"].quantile(0.5)
indus_right_q = new_df["indus"].quantile(0.75)

indus_first_half = new_df["indus"][:len(new_df) // 2].fill_null(indus_left_q)
indus_second_half = new_df["indus"][len(new_df) // 2:].fill_null(indus_right_q)

new_df = new_df.with_columns(
    pl.concat([indus_first_half, indus_second_half])
)

new_df.describe()


statistic,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",500.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,501.0,506.0,506.0
"""null_count""",6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
"""mean""",3.652345,11.363636,11.074684,0.06917,0.554296,6.283864,68.669763,3.782762,9.549407,408.171937,18.454743,356.6002,12.659822,22.549407
"""std""",8.645644,23.322453,6.772542,0.253994,0.114837,0.702596,28.082658,2.098075,8.707259,168.320157,2.159874,91.563468,7.128057,9.182677
"""min""",0.00632,0.0,0.46,0.0,0.385,3.561,2.9,1.1296,1.0,187.0,12.6,0.32,1.73,5.0
"""25%""",0.08199,0.0,5.19,0.0,0.449,5.885,45.1,2.1007,4.0,280.0,17.4,375.33,7.01,17.1
"""50%""",0.26169,0.0,9.69,0.0,0.538,6.205,77.7,3.1827,5.0,330.0,19.1,391.43,11.36,21.2
"""75%""",3.69311,12.5,18.1,0.0,0.624,6.625,94.0,5.118,24.0,666.0,20.2,396.23,16.96,25.0
"""max""",88.9762,100.0,27.74,1.0,0.871,8.78,100.0,12.1265,24.0,711.0,22.0,396.9,37.97,50.0


crim 和 b 沒想到怎麼處理，就線性補值吧。

In [12]:
new_df = new_df.interpolate()

new_df.describe()

statistic,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",3.613756,11.363636,11.074684,0.06917,0.554296,6.283864,68.669763,3.782762,9.549407,408.171937,18.454743,356.866907,12.659822,22.549407
"""std""",8.601465,23.322453,6.772542,0.253994,0.114837,0.702596,28.082658,2.098075,8.707259,168.320157,2.159874,91.152542,7.128057,9.182677
"""min""",0.00632,0.0,0.46,0.0,0.385,3.561,2.9,1.1296,1.0,187.0,12.6,0.32,1.73,5.0
"""25%""",0.08199,0.0,5.19,0.0,0.449,5.885,45.1,2.1007,4.0,280.0,17.4,375.33,7.01,17.1
"""50%""",0.25915,0.0,9.69,0.0,0.538,6.205,77.7,3.1827,5.0,330.0,19.1,391.34,11.36,21.2
"""75%""",3.67822,12.5,18.1,0.0,0.624,6.625,94.0,5.118,24.0,666.0,20.2,396.21,16.96,25.0
"""max""",88.9762,100.0,27.74,1.0,0.871,8.78,100.0,12.1265,24.0,711.0,22.0,396.9,37.97,50.0


最終清理結果：

In [13]:
new_df

crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.09,1.0,296.0,15.3,396.9,4.98,24.0
0.016805,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.9,9.14,21.6
0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.9,5.33,36.2
…,…,…,…,…,…,…,…,…,…,…,…,…,…
0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
0.04527,0.0,11.93,0.0,0.573,6.12,76.7,2.2875,1.0,273.0,21.0,396.9,9.08,20.6
0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.9,5.64,23.9


## 比較執行結果

In [14]:
original_df = pl.read_csv("BostonHousing.csv")

original_df.describe()

statistic,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",3.613524,11.363636,11.136779,0.06917,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
"""std""",8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.10571,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
"""min""",0.00632,0.0,0.46,0.0,0.385,3.561,2.9,1.1296,1.0,187.0,12.6,0.32,1.73,5.0
"""25%""",0.08199,0.0,5.19,0.0,0.449,5.885,45.0,2.1,4.0,279.0,17.4,375.33,6.93,17.0
"""50%""",0.25915,0.0,9.69,0.0,0.538,6.209,77.7,3.2157,5.0,330.0,19.1,391.45,11.38,21.2
"""75%""",3.67822,12.5,18.1,0.0,0.624,6.625,94.1,5.2119,24.0,666.0,20.2,396.23,16.96,25.0
"""max""",88.9762,100.0,27.74,1.0,0.871,8.78,100.0,12.1265,24.0,711.0,22.0,396.9,37.97,50.0


In [15]:
new_df.describe()

statistic,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0,506.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",3.613756,11.363636,11.074684,0.06917,0.554296,6.283864,68.669763,3.782762,9.549407,408.171937,18.454743,356.866907,12.659822,22.549407
"""std""",8.601465,23.322453,6.772542,0.253994,0.114837,0.702596,28.082658,2.098075,8.707259,168.320157,2.159874,91.152542,7.128057,9.182677
"""min""",0.00632,0.0,0.46,0.0,0.385,3.561,2.9,1.1296,1.0,187.0,12.6,0.32,1.73,5.0
"""25%""",0.08199,0.0,5.19,0.0,0.449,5.885,45.1,2.1007,4.0,280.0,17.4,375.33,7.01,17.1
"""50%""",0.25915,0.0,9.69,0.0,0.538,6.205,77.7,3.1827,5.0,330.0,19.1,391.34,11.36,21.2
"""75%""",3.67822,12.5,18.1,0.0,0.624,6.625,94.0,5.118,24.0,666.0,20.2,396.21,16.96,25.0
"""max""",88.9762,100.0,27.74,1.0,0.871,8.78,100.0,12.1265,24.0,711.0,22.0,396.9,37.97,50.0


正確結果看來都在誤差值之內，還行。